In [ ]:
# Import our stuff
from mfpandas import IRRDBU00,DCOLLECT,SETROPTS
# Load them up
racfdb = IRRDBU00(irrdbu00='test_irrdbu00_data')
racfdb.parse_fancycli()
dcol   = DCOLLECT(dcollect='test_dcollect_data')
dcol.parse_fancycli()
setropts = SETROPTS(setropts='test_setropts_data')

In [ ]:
# lets prepare for finding findings ;)
findings = []
def addFind(severity=None, desc=None):
    nr = str(len(findings)+1)
    findings.append([str(nr), severity, desc])



In [ ]:
# Are there any 'UACC read datasets'?
uaccread = racfdb.datasets.loc[racfdb.datasets.DSBD_UACC=='READ']['DSBD_NAME'].values
for f in uaccread:
    addFind(severity="Medium", desc=f'{f} has universal access of READ')
findings

In [ ]:
# access to BPX.SUPERUSER?
who = racfdb.generalAccess.loc[racfdb.generalAccess.GRACC_NAME=='BPX.SUPERUSER'][['GRACC_AUTH_ID','GRACC_ACCESS']]
for w in who.values:
    addFind('HIGH',f'{w[0]} has {w[1]} on BPX.SUPERUSER')

In [ ]:
setropts.fieldInfo.values
revokedAfter = setropts.fieldInfo.loc[setropts.fieldInfo.Setting=='REVOKE']['Value'].values[0]
if revokedAfter == 'FALSE':
    addFind('High', 'UserIDs are not being revoked after invalid password')

In [ ]:
# Prep reporting
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from datetime import datetime
import matplotlib.pyplot as plt
import os



# Count findings per severity
severity_counts = {}
for _, severity, _ in findings:
    severity_counts[severity] = severity_counts.get(severity, 0) + 1

# Generate bar chart
def create_barchart(filename):
    severities = list(severity_counts.keys())
    counts = [severity_counts[s] for s in severities]

    plt.figure(figsize=(6, 4))
    plt.bar(severities, counts, color='skyblue')
    plt.title('Findings per Severity')
    plt.xlabel('Severity')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Create PDF
def create_pdf(output_filename):
    doc = SimpleDocTemplate(output_filename, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    # --- Front Page ---
    story.append(Spacer(1, 200))
    story.append(Paragraph("<font size=24><b>Example Security Audit Report</b></font>", styles["Title"]))
    story.append(Spacer(1, 50))
    story.append(Paragraph("Author: GSNL Session", styles["Title"]))
    story.append(Paragraph("Date: " + datetime.today().strftime('%B %d, %Y'), styles["Title"]))
    story.append(PageBreak())

    # --- Table Page ---
    table_data = [['#', 'Severity', 'Description']] + findings
    table = Table(table_data, colWidths=[40, 130, 380])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    story.append(Paragraph("<b>Audit Findings</b>", styles["Heading2"]))
    story.append(Spacer(1, 12))
    story.append(table)
    story.append(PageBreak())

    # --- Chart Page ---
    chart_file = "barchart.png"
    create_barchart(chart_file)
    story.append(Paragraph("<b>Findings Summary</b>", styles["Heading2"]))
    story.append(Spacer(1, 12))
    story.append(Image(chart_file, width=400, height=300))
    
    doc.build(story)

    # Clean up image
    if os.path.exists(chart_file):
        os.remove(chart_file)

# Run the function
os.remove("audit_report.pdf")
create_pdf("audit_report.pdf")
print('Done')